In [ ]:
from unstructured.partition.pdf import partition_pdf

output_path = "C:/Users/al4k9/OneDrive/Desktop/RAG/RRMenu.pdf"

chunks = partition_pdf(
    filename=output_path,                       # only required parameter
    infer_table_structure=False,                # extract tables
    strategy="auto",                            # mandatory to infer tables, "high-res" if you extract tables
    languages=["eng"],
    # extract_image_block_types=["Image"],      # add 'Table' to list to extrac image of table
    # extract_image_block_output_dir= "path",   # if none, images and tables will be saved in base64


    # chunking groups related information | Useful for RAG
    chunking_strategy="by_title",               # or "basic"
    max_characters=10000,                       # max size of chunk
    combine_text_under_n_chars=2000,            # comebine different elements when they are under 2000 characters
    new_after_n_chars=6000                      # start new part after 6000 characters
    )

In [6]:
print(len(chunks))

9


In [8]:
chunks[0].to_dict()

{'type': 'CompositeElement',
 'element_id': '43c2099fd4e91cc48e55834faccbbcbd',
 'text': 'GOURMET FIRE-GRILLED BURGERS\n\nALL BURGERS INCLUDE A BOTTOMLESS SIDE\n\nSteak Fries • Yukon Chips • Steamed Broccoli • Side Salad Sweet Potato Fries (1.79) • Garlic Fries (1.79)\n\nOUR SIGNATURES Burgers you won’t find anywhere else. 100% Fresh, Never Frozen Beef.\n\nTHE SOUTHERN CHARM BURGER® Brown sugar glaze, candied bacon, Whiskey River® BBQ Sauce, Cheddar, caramelized onions, lettuce and mayo on a toasted brioche bun. 14.99 cal 1190\n\nBLACK & BLEU\n\nSautéed and blackened portobello mushrooms, caramelized onions, creamy cheese sauce, Bleu cheese crumbles, lettuce and roasted garlic aioli on a toasted brioche bun. 15.39 cal 980\n\nWHISKEY RIVER® BBQ\n\nDIP IT. DUNK IT. DOUSE YOUR BURGER.\n\nWhiskey River® BBQ Sauce, crispy onion straws, Cheddar, lettuce, tomatoes and mayo on a toasted sesame bun. 13.79 cal 1140 THE MADLOVE BURGER A Cheddar-and-Parmesan crisp, Provolone, Swiss, jalapeño relis